In [ ]:
import itertools
import json
import pickle
import re
import shutil
import subprocess
import sys
from pathlib import Path
from typing import Any, Dict

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scri
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from spherical_functions import LM_index as lm

## CCE ABD code

In [ ]:
def moving_average_valid(array,avg_len):
    return np.convolve(array,np.ones(avg_len),'valid')/avg_len

def load_and_pickle(data_path:Path, reload_data:bool=False, data_type:str='abd', options:dict={}):
  if not data_path.exists():
    raise Exception(f"{data_path} does not exist!")

  saved_data_path = data_path.parent/"saved.pkl"
  
  if saved_data_path.exists() and reload_data == False:
    with open(saved_data_path, 'rb') as f:
      saved_data = pickle.load(f)
      print(f"Saved data loaded: {saved_data_path}")
  else:
    saved_data = {}
    if data_type == 'abd':
      saved_data['abd']= scri.create_abd_from_h5(
          file_name=str(data_path),
          file_format="spectrecce_v1",
          **options
        )
      with open(saved_data_path, 'wb') as f:
        pickle.dump(saved_data,f)
      print(f"Data loaded and saved at : {saved_data_path}")

  return saved_data

def load_bondi_constraints(data_path:Path):
  if not data_path.exists():
    raise Exception(f"{data_path} does not exist!")
  saved_data_path = data_path.parent/"saved.pkl"
  if not saved_data_path.exists():
    raise Exception(f"{saved_data_path} does not exist")
  else:
    with open(saved_data_path, 'rb') as f:
      saved_data = pickle.load(f)
      if 'bondi_violation_norms' in saved_data:
        print(f"bondi_violation_norms loaded for {data_path}")
      else:
        print(f"Computing bondi_violation_norms for: {data_path}")
        saved_data['bondi_violation_norms'] = saved_data['abd'].bondi_violation_norms
        with open(saved_data_path, 'wb') as f:
          pickle.dump(saved_data,f)

        print(f"Saved bondi_violation_norms for: {data_path}")
    return saved_data

def add_bondi_constraints(abd_data:dict):
  for key in abd_data:
    abd_data[key]['bondi_violation_norms'] = abd_data[key]["abd"].bondi_violation_norms
    print(f"bondi_violation_norms computed for {key}")

def create_diff_dict_cce(WT_data_dict:dict, l:int, m:int, base_key:str, t_interpolate:np.ndarray):
  h = WT_data_dict[base_key]['abd'].h.interpolate(t_interpolate)
  diff_dict = {"t": h.t}
  y_base = h.data[:,  lm(l,m,h.ell_min)]
  y_norm = np.linalg.norm(y_base)
  for key in WT_data_dict:
    if key == base_key:
      continue
    h = WT_data_dict[key]['abd'].h.interpolate(t_interpolate)
    y_inter = h.data[:,  lm(l,m,h.ell_min)]
    diff_dict[key+"_diff"] = y_inter-y_base
    diff_dict[key+"_absdiff"] = np.abs(y_inter-y_base)
    diff_dict[key+"_rel_diff"] = (y_inter-y_base)/y_norm
    diff_dict[key+"_rel_absdiff"] = np.abs(y_inter-y_base)/y_norm
  return diff_dict

def extract_radii(h5_file_path:Path):
  radii = set()
  with h5py.File(h5_file_path,'r') as f:
    names = []
    f.visit(names.append)
  for name in names:
    if "Version" in name:
      continue
    radii.add(name[1:5])
  radii = list(radii)
  radii.sort()
  return radii


def generate_columns(num_cols:int,beta_type=False):
  if beta_type:
    num_cols = num_cols*2
  L_max = int(np.sqrt((num_cols-1)/2))-1
  # print(L_max,np.sqrt((num_cols-1)/2)-1)
  col_names = ['t(M)']
  for l in range(0,L_max+1):
    for m in range(-l,l+1):
      if beta_type:
        if m==0:
          col_names.append(f"Re({l},{m})")
        elif m < 0:
          continue
        else:
          col_names.append(f"Re({l},{m})")
          col_names.append(f"Im({l},{m})")
      else:
        col_names.append(f"Re({l},{m})")
        col_names.append(f"Im({l},{m})")
  return col_names


def WT_to_pandas(horizon_path:Path):
    assert(horizon_path.exists())
    df_dict = {}
    beta_type_list = ['Beta.dat', 'DuR.dat', 'R.dat', 'W.dat']
    with h5py.File(horizon_path,'r') as hf:
        # Not all horizon files may have AhC
        for key in hf.keys():
            if key == "VersionHist.ver":
              continue 
            if key in beta_type_list:
              df_dict[key] = pd.DataFrame(hf[key], columns=generate_columns(hf[key].shape[1],beta_type=True))
            else:
              df_dict[key] = pd.DataFrame(hf[key], columns=generate_columns(hf[key].shape[1]))


    return df_dict


def create_diff_dict(WT_data_dict:dict, mode:str, variable:str, base_key:str):
  diff_dict = {"t(M)":WT_data_dict[base_key][variable]['t(M)']}
  y_base = WT_data_dict[base_key][variable][mode]
  y_norm = np.linalg.norm(y_base)
  for key in WT_data_dict:
    if key == base_key:
      continue
    y = WT_data_dict[key][variable][mode]
    t = WT_data_dict[key][variable]['t(M)']
    y_interpolator = interp1d(t, y, kind='cubic',fill_value='extrapolate')
    y_inter = y_interpolator(diff_dict['t(M)'])
    diff_dict[key+"_diff"] = y_inter-y_base
    diff_dict[key+"_absdiff"] = np.abs(y_inter-y_base)
    diff_dict[key+"_rel_diff"] = (y_inter-y_base)/y_norm
    diff_dict[key+"_rel_absdiff"] = np.abs(y_inter-y_base)/y_norm
  return diff_dict

def filter_by_regex(regex,col_list,exclude=False):
  filtered_set = set()
  if type(regex) is list:
    for reg in regex:
      for i in col_list:
        if re.search(reg,i):
          filtered_set.add(i)
  else:
    for i in col_list:
      if re.search(regex,i):
        filtered_set.add(i)

  filtered_list = list(filtered_set)
  if exclude:
    col_list_copy = list(col_list.copy())
    for i in filtered_list:
      if i in col_list_copy:
        col_list_copy.remove(i)
    filtered_list = col_list_copy

  # Restore the original order
  filtered_original_ordered_list = []
  for i in list(col_list):
    if i in filtered_list:
      filtered_original_ordered_list.append(i)
  return filtered_original_ordered_list

def limit_by_col_val(min_val,max_val,col_name,df):
  filter = (df[col_name]>=min_val) &(df[col_name] <=max_val)
  return df[filter]

def abs_mean_value_upto_l(pd_series,L_max:int):
  idx = pd_series.index
  abs_cum_sum = 0
  num = 0
  for i in idx:
    L = int(i.split(",")[0][3:])
    if L > L_max:
      continue
    else:
      abs_cum_sum = abs_cum_sum+abs(pd_series[i])
      num = num +1
  return abs_cum_sum/num

def get_mode(name):
  return int(name.split("(")[-1].split(")")[0])
def get_radii(name):
  if name[-5]=='R':
    # R0257 -> 0257load_and_pickle
    return int(name.split('_')[-1][1:])
  else:
    return int(name.split('_')[-1])
def sort_by_power_modes(col_names):
  col_name_copy = list(col_names).copy()
  return sorted(col_name_copy, key=lambda x: int(get_mode(x)))

def add_L_mode_power(df:pd.DataFrame,L:int, ReOrIm:str):
  column_names = df.columns
  n = 0
  power = 0
  for m in range(-L,L+1):
    col_name = f'{ReOrIm}({L},{m})'
    # print(col_name)
    if col_name in column_names:
      power = power + df[col_name]*df[col_name]
      n = n + 1
  if n != 0:
    power = power/n
    df[f'pow_{ReOrIm}({L})'] = power
  return power

def add_all_L_mode_power(df:pd.DataFrame,L_max:int):
  local_df = df.copy()
  total_power_Re = 0
  total_power_Im = 0
  for l in range(0,L_max+1):
    total_power_Re = total_power_Re + add_L_mode_power(local_df,l,"Re")
    total_power_Im = total_power_Im + add_L_mode_power(local_df,l,"Im")
    local_df[f"pow_cum_Re({l})"] = total_power_Re
    local_df[f"pow_cum_Im({l})"] = total_power_Im
  return local_df


def create_power_diff_dict(power_dict:dict, pow_mode:str, variable:str, base_key:str):
  diff_dict = {"t(M)":power_dict[base_key]['t(M)']}
  y_base = power_dict[base_key][variable][pow_mode]
  y_norm = np.linalg.norm(y_base)
  for key in power_dict:
    if key == base_key:
      continue
    y = power_dict[key][variable][pow_mode]
    t = power_dict[key]['t(M)']
    y_interpolator = interp1d(t, y, kind='cubic',fill_value='extrapolate')
    y_inter = y_interpolator(diff_dict['t(M)'])
    diff_dict[key+"_diff"] = y_inter-y_base
    diff_dict[key+"_absdiff"] = np.abs(y_inter-y_base)
    diff_dict[key+"_rel_diff"] = (y_inter-y_base)/y_norm
    diff_dict[key+"_rel_absdiff"] = np.abs(y_inter-y_base)/y_norm
  return diff_dict

In [ ]:
def copy_and_modify_bondi_data(input_file, output_file, eps=1e-12, rand_amp=0):
    shutil.copy(input_file, output_file)
    with h5py.File(output_file, "r+") as outfile:
        for key1, item1 in outfile.items():
            if "Version" in key1:
                continue
            if isinstance(item1, h5py.Dataset):
                print(f"----Modifying {key1}")
                data = outfile[key1][()] + 0
                mask = np.abs(data) < eps
                mask[:, 0] = False  # Time should not be filtered
                data[mask] = rand_amp * (2 * np.random.rand(*data[mask].shape) - 1)
                outfile[key1][()] = data


def create_WT_data_compressed(
    input_file,
    output_file,
    trange=np.arange(0, 10000, 0.1),
    rand_amp=None,
    extraction_radii=100,
):
    shutil.copy(input_file, output_file)
    with h5py.File(output_file, "r+") as outfile:
        for key1, item1 in outfile.items():
            if "Version" in key1:
                continue
            if isinstance(item1, h5py.Dataset):
                print(f"----Modifying {key1}")
                data = np.zeros((len(trange), outfile[key1].shape[1]))
                print(data.shape)
                data[:, 0] = trange
                R_val = 3.544907701811031 * extraction_radii
                if "W.dat" == key1:
                    data[:, 1] = -89.09324794930309 / R_val**2
                if "R.dat" == key1:
                    data[:, 1] = R_val
                if rand_amp is not None:
                    data[:, 1:] += rand_amp * (
                        2 * np.random.rand(*data[:, 1:].shape) - 1
                    )

                # Save all attributes
                attr_dict = {}
                for attr_key, attr_value in outfile[key1].attrs.items():
                    attr_dict[attr_key] = attr_value

                del outfile[key1]
                ds = outfile.create_dataset(
                    key1,
                    data=data,
                    compression="gzip",  # or 'lzf', 'szip'
                    compression_opts=9 if "gzip" else None,  # level 0-9 for gzip
                    chunks=True,  # chunking is required for compression
                )

                # Restore all attributes
                for attr_key, attr_value in attr_dict.items():
                    ds.attrs[attr_key] = attr_value

In [ ]:
def make_config_file(
    BoundaryDataPath: Path,
    InputSavePath: Path = None,
    which_ID: str = "ConformalFactor",
    options_dict_user={},
) -> Path:
    options_dict = {
        "Cce.Evolution.TimeStepper.AdamsBashforth.Order": 3,
        "Cce.Evolution.StepChoosers.Constant": 0.1,
        "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).AbsoluteTolerance": 1e-9,
        "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).RelativeTolerance": 1e-7,
        "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).AbsoluteTolerance": 1e-9,
        "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).RelativeTolerance": 1e-8,
        "Cce.LMax": 25,
        "Cce.NumberOfRadialPoints": 20,
        "Cce.ObservationLMax": 8,
        "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MinOrder": 10,
        "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MaxOrder": 10,
        "Cce.H5LookaheadTimes": 10000,
        "Cce.Filtering.RadialFilterHalfPower": 64,
        "Cce.Filtering.RadialFilterAlpha": 35.0,
        "Cce.Filtering.FilterLMax": 18,
        "Cce.ScriInterpOrder": 5,
        "Cce.ScriOutputDensity": 1,
    }

    for key in options_dict_user.keys():
        if key not in options_dict:
            raise ValueError(f"Key {key} is not a valid option.")
        else:
            options_dict[key] = options_dict_user[key]

    CCE_ID_data = ""
    match which_ID:
        case "ConformalFactor":
            CCE_ID_data = """
    ConformalFactor:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000 # Do extra iterations in case we improve.
      RequireConvergence: False # Often don't converge to 1e-13, but that's fine
      OptimizeL0Mode: True
      UseBetaIntegralEstimate: False
      ConformalFactorIterationHeuristic: SpinWeight1CoordPerturbation
      UseInputModes: False
      InputModes: []
"""
        case "InverseCubic":
            CCE_ID_data = """
    InverseCubic:
"""
        case "ZeroNonSmooth":
            CCE_ID_data = """
    ZeroNonSmooth:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000
      RequireConvergence: False
"""
        case "NoIncomingRadiation":
            CCE_ID_data = """
    NoIncomingRadiation:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000
      RequireConvergence: False
"""
    if InputSavePath is None:
        InputSavePath = BoundaryDataPath.parent / "cce.yaml"
    assert InputSavePath.parent.exists()

    config_file = f"""
# Distributed under the MIT License.
# See LICENSE.txt for details.

# This block is used by testing and the SpECTRE command line interface.
Executable: CharacteristicExtract
Testing:
  Check: parse
  Priority: High

---
Evolution:
  InitialTimeStep: 0.25
  InitialSlabSize: 10.0

ResourceInfo:
  AvoidGlobalProc0: false
  Singletons: Auto

Observers:
  VolumeFileName: "vol_{str(InputSavePath.stem)}"
  ReductionFileName: "red_{str(InputSavePath.stem)}"

EventsAndTriggers:
  # Write the CCE time step every Slab. A Slab is a fixed length of simulation
  # time and is not influenced by the dynamically adjusted step size.
  - Trigger:
      Slabs:
        EvenlySpaced:
          Offset: 0
          Interval: 1
    Events:
      - ObserveTimeStep:
          # The output is written into the "ReductionFileName" HDF5 file under
          # "/SubfileName.dat"
          SubfileName: CceTimeStep
          PrintTimeToTerminal: true

Cce:
  Evolution:
    TimeStepper:
      AdamsBashforth:
        Order: {options_dict['Cce.Evolution.TimeStepper.AdamsBashforth.Order']} # Going to higher order doesn't seem necessary for CCE
    StepChoosers:
      - Constant: {options_dict['Cce.Evolution.StepChoosers.Constant']} # Don't take steps bigger than 0.1M
      - Increase:
          Factor: 2
      - ErrorControl(SwshVars):
          AbsoluteTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(SwshVars).AbsoluteTolerance']}
          RelativeTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(SwshVars).RelativeTolerance']}
          # These factors control how much the time step is changed at once.
          MaxFactor: 2
          MinFactor: 0.25
          # How close to the "perfect" time step we take. Since the "perfect"
          # value assumes a linear system, we need some safety factor since our
          # system is nonlinear, and also so that we reduce how often we retake
          # time steps.
          SafetyFactor: 0.9
      - ErrorControl(CoordVars):
          AbsoluteTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(CoordVars).AbsoluteTolerance']}
          RelativeTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(CoordVars).RelativeTolerance']}
          # These factors control how much the time step is changed at once.
          MaxFactor: 2
          MinFactor: 0.25
          # How close to the "perfect" time step we take. Since the "perfect"
          # value assumes a linear system, we need some safety factor since our
          # system is nonlinear, and also so that we reduce how often we retake
          # time steps.
          SafetyFactor: 0.9

  # The number of angular modes used by the CCE evolution. This must be larger
  # than ObservationLMax. We always use all of the m modes for the LMax since
  # using fewer m modes causes aliasing-driven instabilities.
  LMax: {options_dict['Cce.LMax']}
  # Probably don't need more than 15 radial grid points, but could increase
  # up to ~20
  NumberOfRadialPoints: {options_dict['Cce.NumberOfRadialPoints']}
  # The maximum ell we use for writing waveform output. While CCE can dump
  # more, you should be cautious with higher modes since mode mixing, truncation
  # error, and systematic numerical effects can have significant contamination
  # in these modes.
  ObservationLMax: {options_dict['Cce.ObservationLMax']}

  InitializeJ:
    # To see what other J-initialization procedures are available, comment
    # out this group of options and do, e.g. "Blah:" The code will print
    # an error message with the available options and a help string.
    # More details can be found at spectre-code.org.
{CCE_ID_data}

  StartTime: Auto
  EndTime: Auto
  ExtractionRadius: Auto

  BoundaryDataFilename: {BoundaryDataPath.name}
  H5IsBondiData: True
  H5Interpolator:
    BarycentricRationalSpanInterpolator:
      MinOrder: {options_dict['Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MinOrder']}
      MaxOrder: {options_dict['Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MaxOrder']}
  FixSpecNormalization: False

  H5LookaheadTimes: {options_dict['Cce.H5LookaheadTimes']}

  Filtering:
    RadialFilterHalfPower: {options_dict['Cce.Filtering.RadialFilterHalfPower']}
    RadialFilterAlpha: {options_dict['Cce.Filtering.RadialFilterAlpha']}
    FilterLMax: {options_dict['Cce.Filtering.FilterLMax']}

  ScriInterpOrder: {options_dict['Cce.ScriInterpOrder']}
  ScriOutputDensity: {options_dict['Cce.ScriOutputDensity']}

"""

    with InputSavePath.open("w") as f:
        f.writelines(config_file)

    return InputSavePath


def make_submit_file(
    save_folder_path: Path,
    cce_input_file_path: Path,
    CCE_Executable_path: Path,
    write_scripts_only=False,
):
    submit_script = f"""#!/bin/bash -
#SBATCH -J CCE_{save_folder_path.stem}             # Job Name
#SBATCH -o CCE.stdout                 # Output file name
#SBATCH -e CCE.stderr                 # Error file name
#SBATCH -n 2                          # Number of cores
#SBATCH -p expansion                  # Queue name
#SBATCH --ntasks-per-node 2           # number of MPI ranks per node
#SBATCH -t 24:0:00   # Run time
#SBATCH -A sxs                # Account name
#SBATCH --no-requeue
#SBATCH --reservation=sxs_standing

# Go to the correct folder with the boundary data
cd {save_folder_path}

# run CCE
{CCE_Executable_path} --input-file ./{cce_input_file_path.name}
"""
    submit_script_path = save_folder_path / "submit.sh"
    submit_script_path.write_text(submit_script)

    if not write_scripts_only:
        command = f"cd {save_folder_path} && qsub {submit_script_path}"
        status = subprocess.run(command, capture_output=True, shell=True, text=True)
        if status.returncode == 0:
            print(f"Succesfully submitted {submit_script_path}\n{status.stdout}")
        else:
            sys.exit(
                f"Job submission failed for {submit_script_path} with error: \n{status.stdout} \n{status.stderr}"
            )

In [ ]:
def create_CCE_single_bh(
    NewCCEPath: Path,
    BondiBaseH5File: Path,
    CCE_Executable_path: Path,
    ExtractionRadii_str: str,
    CCE_ID: str = "ConformalFactor",
    options_dict_user={},
    write_scripts_only=False,
    trange=np.arange(0, 10000, 0.1),
    rand_amp=None,
):
    NewCCEPath = NewCCEPath.resolve()
    BondiBaseH5File = BondiBaseH5File.resolve()
    CCE_Executable_path = CCE_Executable_path.resolve()

    if not BondiBaseH5File.exists():
        raise Exception(f"{BondiBaseH5File} does not exist!")
    if not CCE_Executable_path.exists():
        raise Exception(f"{CCE_Executable_path} does not exist!")

    NewCCEPath.mkdir(parents=True, exist_ok=False)

    create_WT_data_compressed(
        BondiBaseH5File,
        NewCCEPath / f"BondiDataR{ExtractionRadii_str}.h5",
        trange=trange,
        rand_amp=rand_amp,
        extraction_radii=int(ExtractionRadii_str),
    )

    make_config_file(
        BoundaryDataPath=NewCCEPath / f"BondiDataR{ExtractionRadii_str}.h5",
        InputSavePath=NewCCEPath / f"cce.yaml",
        which_ID=CCE_ID,
        options_dict_user=options_dict_user,
    )

    make_submit_file(
        save_folder_path=NewCCEPath,
        cce_input_file_path=NewCCEPath / f"cce.yaml",
        CCE_Executable_path=CCE_Executable_path,
        write_scripts_only=write_scripts_only,
    )

    print("DONE!\n\n\n\n\n")


In [ ]:
# create_CCE_single_bh(
#     NewCCEPath = Path("./test_CCE"),
#     BondiBaseH5File = Path("/media/himanshu/T7_500g/CCE_data/all_bondi_data/BondiCceR0100.h5"),
#     CCE_Executable_path = Path("/media/himanshu/T7_500g/CCE_data/CCE_executable/CharacteristicExtract"),
#     ExtractionRadii_str='0100',
#     CCE_ID="ConformalFactor",
#     write_scripts_only=True,
#     trange=np.arange(0, 1000, 0.1),
# )

## Other code

In [ ]:
def read_h5_file_dump_tensors(file_path: str) -> Dict[str, Any]:
    def read_group(group) -> Dict[str, Any]:
        result = {}

        # Check if group is actually a group before iterating
        if isinstance(group, h5py.Group):
            # Read all datasets in current group
            for name, item in group.items():
                if isinstance(item, h5py.Dataset):
                    # Convert dataset to numpy array
                    result[name] = item[()]
                elif isinstance(item, h5py.Group):
                    # Recursively read nested group
                    result[name] = read_group(item)
                else:
                    print(name, item)

            # Read attributes
            for name, item in group.attrs.items():
                result[name] = item
        elif isinstance(group, h5py.Dataset):
            # If it's a dataset, just return its value
            return group[()]

        return result

    # Check if file exists
    if not Path(file_path).exists():
        raise FileNotFoundError(f"The file {file_path} does not exist")
    try:
        with h5py.File(file_path, "r") as f:
            # Read all contents
            data = {}
            # Read main groups
            for name, item in f.items():
                data[name] = read_group(f[name])
            # # Read main groups
            # for group_name in ["InitGridHi", "InitHhatt", "kappa", "psi"]:
            #     if group_name in f:
            #         data[group_name] = read_group(f[group_name])
        return data

    except OSError as e:
        raise OSError(f"Error reading HDF5 file: {str(e)}")


def create_modification_dict(
    filter_modes_output_path: Path, filtered_vars_folders_path: Path
):

    filtered_sub_domains = set()
    with filter_modes_output_path.open() as f:
        for l in f.readlines():
            if re.match(r"Filtered=", l):
                domain_name = l.split("=")[-1].strip()
                filtered_sub_domains.add(domain_name)

    filtered_dict = {}
    vars_h5_files = list(filtered_vars_folders_path.glob("Vars_*.h5"))
    # load data for subdomains that were filtered
    for fp in vars_h5_files:
        domain_name = fp.stem.split("_")[-1]
        if domain_name in filtered_sub_domains:
            filtered_dict[domain_name] = read_h5_file_dump_tensors(fp)

    return filtered_dict


def copy_and_modify_h5file(input_file, output_file, modification_data_dict):
    shutil.copy(input_file, output_file)
    with h5py.File(output_file, "r+") as outfile:
        # Level 1: Root level items
        for key1, item1 in outfile.items():
            if isinstance(item1, h5py.Group):
                # Level 2: First nested level
                for key2, item2 in item1.items():
                    if isinstance(item2, h5py.Group):
                        # Level 3: Second nested level
                        for key3, item3 in item2.items():
                            if isinstance(item3, h5py.Dataset):
                                # print(f"{key1}/{key2}/{key3}")
                                if key1 in modification_data_dict:
                                    print(f"----Modifying {key1}/{key3}")
                                    outfile[key1][key2][key3][()] = (
                                        modification_data_dict[key1][key2][key3]
                                    )
                            else:
                                raise ValueError(f"Unexpected item type: {type(item3)}")
                    else:
                        raise ValueError(f"Unexpected item type: {type(item2)}")
            else:
                raise ValueError(f"Unexpected item type: {type(item1)}")


def make_filtered_checkpoints(
    checkpoint_folder: Path, new_checkpoint_folder: Path, data_dict: dict
):
    # Copy the original checkpoint file
    for fp in checkpoint_folder.glob("*.txt"):
        shutil.copy(fp, new_checkpoint_folder)

    for fp in checkpoint_folder.glob("*.h5"):
        file_name = fp.stem
        # get domain name for the h5 files
        if file_name == "SerialCheckpoint":
            # Copy this without change
            shutil.copy(fp, new_checkpoint_folder)
            continue

        domain_name = file_name.split("_")[-1]

        if domain_name in data_dict:
            print(f"Modifying the domain {domain_name}")
            modified_data = data_dict[domain_name]
            modified_fp = new_checkpoint_folder / fp.name
            copy_and_modify_h5file(fp, modified_fp, modified_data)
        else:
            print(f"Copying the domain {domain_name}")
            shutil.copy(fp, new_checkpoint_folder)


def make_input_file_content(half_power_R, half_power_L):
    input_file_text = f"""DataBoxItems =
        ReadFromFile(File=./SpatialCoordMap.input),
        ReadFromFile(File=./GaugeItems.input),
        Domain(Items=
            AddGeneralizedHarmonicInfo(MatterSourceName=;)
            ),
        Subdomain(Items =
                Add3Plus1ItemsFromGhPsiKappa(psi=psi;kappa=kappa;OutputPrefix=),
                AddSpacetimeJacobianAndHessianItems(MapPrefix=GridToInertial;),
                GlobalDifferentiator
                (GlobalDifferentiator=
                    MatrixMultiply(MultiDim_by_BasisFunction=yes;
                    TopologicalDifferentiator
                    =Spectral(SetBasisFunctionsFromTimingInfo=yes;
                            # BasisFunctions= (ChebyshevGaussLobatto=ChebyshevGaussLobattoMatrix);
                            )
                    );
                ),"""
    for R, L, i in zip(half_power_R, half_power_L, np.arange(len(half_power_R))):
        input_file_text = (
            input_file_text
            + f"""
    # ==============================================================================
                FilterVar(
                    Input = psi;
                    Output = fil_psi{i};
                    WhichFilter = Exp;
                    RadialHalfPower = {R};
                    AngularHalfPower = {L};
                    DomainRegex = SphereC{i};
                ),
                FilterVar(
                    Input = kappa;
                    Output = fil_kappa{i};
                    WhichFilter = Exp;
                    RadialHalfPower = {R};
                    AngularHalfPower = {L};
                    DomainRegex = SphereC{i};
                ),
                FirstDeriv(
                    Input = fil_psi{i};
                    Output = fil_dpsi_TT{i};
                    # SetDerivDimFromTensorDim=True;
                    MapPrefix = GridToInertial;
                ),
                FlattenDeriv(
                    Input = fil_dpsi_TT{i};
                    Output = fil_dpsi{i};
                    DerivPosition = First;
                    ZeroFillOffset=1;
                ),
                FakeKappa(
                    FildPsi = fil_dpsi{i};
                    FilKappa = fil_kappa{i};
                    Output = fake_kappa{i};
                ),
"""
        )

    input_file_text = (
        input_file_text
        + """


    ;);#Subdomains
    Observers ="""
    )

    for R, L, i in zip(half_power_R, half_power_L, np.arange(len(half_power_R))):
        input_file_text = (
            input_file_text
            + f"""        DumpTensors(
        Input = fil_psi{i},fake_kappa{i};
        FileNames = psi,kappa;
        OnlyTheseSubdomains = SphereC{i};
        ),        
        ObserveInSubdir
        (Subdir=SpecCoeffs;
          Observers =
          DumpSpectralCoefficients(
            Input = fake_kappa{i}, fil_psi{i}, fil_kappa{i}, fil_dpsi{i};
            TakeLog=no;
            Subdomains =SphereC{i};
            # Eps = 1e-16;
          ),
        ),
"""
        )

    input_file_text = (
        input_file_text
        + """
        ObserveInSubdir
        (Subdir=SpecCoeffs;
          Observers =
          DumpSpectralCoefficients(
            Input = psi, kappa;
            TakeLog=no;
            Subdomains = *;
            # Eps = 1e-16;
          ),
        ),
        ;
    """
    )
    return input_file_text


def make_filtered_checkpoint_from_another(
    ApplyObserversPath: Path,
    InputAndHistFolderPath: Path,
    CheckpointFolderPath: Path,
    work_dir: Path,
    HalfPowerArr: np.ndarray,
):

    if not CheckpointFolderPath.exists():
        raise Exception(f"{CheckpointFolderPath} does not exist!!")

    filtered_checkpoint_path = work_dir / "filtered_checkpoint"
    filtered_text_files_path = work_dir / "data"

    work_dir.mkdir(parents=True, exist_ok=True)
    filtered_checkpoint_path.mkdir(parents=False, exist_ok=True)
    filtered_text_files_path.mkdir(parents=False, exist_ok=True)

    for f in CheckpointFolderPath.glob("*"):
        shutil.copy(f, work_dir)
    for f in InputAndHistFolderPath.glob("Hist*.txt"):
        shutil.copy(f, work_dir)
    for f in InputAndHistFolderPath.glob("*.input"):
        shutil.copy(f, work_dir)

    apply_observer = f"""#!/bin/bash
. /home/hchaudha/spec/MakefileRules/this_machine.env
cd {work_dir}

{ApplyObserversPath} -t psi,kappa,InitHhatt,InitGridHi -r 11,122,,1 -d 4,4,1,3 -domaininput ./GrDomain.input -h5prefix Cp-VarsGr ./input_file.input
    """
    with open(work_dir / "input_file.input", "w") as f:
        f.write(make_input_file_content(HalfPowerArr, HalfPowerArr))

    with open(work_dir / "apply_observer.sh", "w") as f:
        f.write(apply_observer)

    command = f"cd {work_dir} && bash ./apply_observer.sh > ./apply_observer.out"
    apply_observer_output_path = work_dir / "apply_observer.out"

    status = subprocess.run(command, capture_output=True, shell=True, text=True)
    if status.returncode == 0:
        print(f"Ran FilterModes observer in {work_dir}.\n {status.stdout}")
    else:
        sys.exit(
            f"Failed to run FilterModes observer in {work_dir}. \n {status.stderr}"
        )

    # load the filtered data in txt format into a dictionary
    filtered_data_dict = create_modification_dict(
        filter_modes_output_path=apply_observer_output_path,
        filtered_vars_folders_path=work_dir,
    )
    print(
        f"Loaded filtered data for the following domains: {filtered_data_dict.keys()}"
    )

    # create new checkpoint files with the filtered data
    make_filtered_checkpoints(
        CheckpointFolderPath, filtered_checkpoint_path, filtered_data_dict
    )

    # Rename the original checkpoint folder
    new_original_checkpoint_folder_name = (
        CheckpointFolderPath.parent / f"{CheckpointFolderPath.stem}_original"
    )
    shutil.move(CheckpointFolderPath, new_original_checkpoint_folder_name)

    # Copy the filtered checkpoint data into the place of the original checkpoint data
    shutil.copytree(work_dir / "filtered_checkpoint", CheckpointFolderPath)

    # Copy the main file for reproducibility
    shutil.copy(Path(__file__).absolute(), work_dir, follow_symlinks=True)

In [ ]:
# data = {}
# for file in Path("./all_bondi_data").glob("BondiCceR*"):
#     new_data = read_h5_file_dump_tensors(Path(file))
#     W_max = np.max(np.abs(new_data['W.dat'][:,1:]))
#     R_max = np.max(np.abs(new_data['R.dat'][:,1:]))
#     key = file.stem
#     data[key] = {'W': W_max, 'R': R_max}
#     print(key, W_max, R_max)

# for key in data.keys():
#     radii = float(key.split("R")[-1])
#     print(key, data[key]['R']/radii)

## abd stuff

In [ ]:
cce_data = {}
# cce_data[f"ode_tol_10"] = Path(f"/central/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/r100_data/ode_10/red_cce.h5")
# cce_data[f"ode_tol_100"] = Path(f"/central/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/r100_data/ode_100/red_cce.h5")
# cce_data[f"ode_tol_1000"] = Path(f"/central/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/r100_data/ode_1000/red_cce.h5")
# cce_data[f"ode_tol_10000"] = Path(f"/central/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/r100_data/ode_10000/red_cce.h5")
cce_data[f"test"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh_CCE/runs/test1/LMax10_RadPts10/red_cce.h5")


fail_flag=False
for key in cce_data:
  if not cce_data[key].exists():
    fail_flag = True
    print(f"{cce_data[key]} does not exist!")
  if fail_flag:
    raise Exception("Some paths do not exist!")

In [ ]:
t_interpolate = np.linspace(-1000,12000,num=2000)
abd_data = {}
failed_keys=  {}
for key in cce_data:
  try:
    abd_data[key] = load_and_pickle(cce_data[key], options={'t_interpolate': t_interpolate})
    abd_data[key] = load_bondi_constraints(cce_data[key])
  except Exception as e:
    failed_keys[key] = str(e)
    print(f"Failed to load and pickle data for key {key}: {e}")
    continue

print(abd_data.keys())

In [ ]:
bondi_norms_to_plot = [0,1,2,3,4,5]
bondi_norms_to_plot = [2]

t_min = -10000
# t_min = 2000
# t_min = 2500
# t_min = 7500
# t_min = 4000 - 250
t_max = 10000
# t_max = 7751.5
labels = None

def get_radius(run_name):
  radius_part = run_name.split("_")[-1]
  a = "".join([i for i in radius_part if i.isdigit()])
  return int(a)

min_radius = 0
min_radius = 260
max_radius = 1000
max_radius = 360

y_list = []
for key in abd_data:
  violation_dict = abd_data[key]['bondi_violation_norms']
#   radius = get_radius(key)
#   if radius > max_radius or radius < min_radius:
#     continue
  # if "Factor" in key:
  #   continue

  t_arr = abd_data[key]["abd"].t
  trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
  t_arr = t_arr[trimmed_indices]

  for i in bondi_norms_to_plot:
    if labels is None:
      plt.semilogy(t_arr,violation_dict[i][trimmed_indices],label=f"{key}_{i}")
      # plt.semilogy(t_arr,violation_dict[i][trimmed_indices])
    else:
      plt.semilogy(t_arr,violation_dict[i][trimmed_indices],label=f"{labels[key]}_{i}")

  plt.xlabel('t(M)')
  plt.ylabel(f"bondi violations {bondi_norms_to_plot}")
  plt.legend()
  plt.tight_layout()
#   plt.show()
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")

In [ ]:
bondi_norms_to_plot = [0,1,2,3,4,5]
bondi_norms_to_plot = [2]

t_min = -10000
# t_min = 1200
# t_min = 2500
# t_min = 7500
# t_min = 4000 - 250
t_max = 10000
# t_max = 7751.5
# t_max = 1200
# t_max = 3600
# t_max = 4000
# t_max = 7000

labels = None
# labels={
# #  "high_accuracy_Lev5_R0258" : "Lev5_ode_tol_R0258",
#  "6_set1_L3s3_200" : "Lev3_variant_R0200",
#  "6_set1_L3s3_250" : "Lev3_variant_R0250",
#  "6_set1_L3s3_300" : "Lev3_variant_R0300",
#  "6_set1_L3s2_200" : "Lev2_variant_R0200",
#  "6_set1_L3s2_250" : "Lev2_variant_R0250",
#  "6_set1_L3s2_300" : "Lev2_variant_R0300",
# }

def get_radius(run_name):
  radius_part = run_name.split("_")[-1]
  a = "".join([i for i in radius_part if i.isdigit()])
  return int(a)

min_radius = 0
# min_radius = 260
max_radius = 1000
# max_radius = 360

y_list = []
for key in abd_data:
#   violation_dict = abd_data[key]['bondi_violation_norms']
#   radius = get_radius(key)
#   if radius > max_radius or radius < min_radius:
#     continue
  t_arr = abd_data[key]["abd"].t
  trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
  t_arr = t_arr[trimmed_indices]

  print(key)
  violation_dict = abd_data[key]['abd'].bondi_rest_mass() - 1.0
  plt.plot(t_arr,violation_dict[trimmed_indices],label=f"{key}")


#   violation_dict = abd_data[key]['abd'].bondi_angular_momentum()
#   plt.plot(t_arr,violation_dict[:,0][trimmed_indices],label=f"{key}_0")
#   plt.plot(t_arr,violation_dict[:,1][trimmed_indices],label=f"{key}_1")
#   plt.plot(t_arr,violation_dict[:,2][trimmed_indices],label=f"{key}_2")

  plt.xlabel('t(M)')
#   plt.ylabel(f"bondi violations ")
  plt.legend()
  plt.tight_layout()
#   plt.yscale('log')
  # plt.show()
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")

## Rough

In [ ]:
WT_data = Path("./BondiCceR0100/BondiCceR0100.h5")
if not WT_data.exists():
    raise Exception(f"{WT_data} does not exist!!")

In [ ]:
data = read_h5_file_dump_tensors(WT_data)

In [ ]:
for key in data.keys():
    if "Version" in key:
        continue
    print(key, np.max(np.abs(data[key][:,1:])))


In [ ]:
for key in data.keys():
    if "Version" in key:
        continue
    print(key, np.max(data[key][:,1:]))

In [ ]:
copy_and_modify_bondi_data(WT_data, WT_data.parent/"new.h5", eps=1e-12, rand_amp=0)

In [ ]:
new_data = read_h5_file_dump_tensors(WT_data.parent/"new.h5")

In [ ]:
# new_data = read_h5_file_dump_tensors(Path("/media/himanshu/T7_500g/CCE_data/BondiCceR0100/red_cce.h5"))
new_data = read_h5_file_dump_tensors(Path("/media/himanshu/T7_500g/CCE_data/test/red_cce.h5"))
cce_key = list(new_data.keys())[1]
print(cce_key)

In [ ]:
y = new_data[cce_key]['Strain'][:,1:]
# y = new_data[cce_key]['Psi0'][:,1:]
# y = new_data[cce_key]['Psi1'][:,1:]
# y = new_data[cce_key]['Psi2'][:,1:]
# y = new_data[cce_key]['Psi3'][:,1:]
# y = new_data[cce_key]['Psi4'][:,1:]


In [ ]:
# new_data = read_h5_file_dump_tensors(Path("/media/himanshu/T7_500g/CCE_data/Set1L6/BondiCceR0300.h5"))
new_data = read_h5_file_dump_tensors(Path("all_bondi_data/BondiCceR0020.h5"))
# new_data = read_h5_file_dump_tensors(Path("/media/himanshu/T7_500g/CCE_data/test/red_cce.h5"))
print(new_data.keys())

In [ ]:
y = new_data['Beta.dat'][:,1:]
y = new_data['DrJ.dat'][:,1:]
y = new_data['DuR.dat'][:,1:]
y = new_data['H.dat'][:,1:]
# y = new_data['J.dat'][:,1:]
# y = new_data['Q.dat'][:,1:]
y = new_data['R.dat'][:,1:]
# y = new_data['U.dat'][:,1:]
# y = new_data['W.dat'][:,1:]

In [ ]:
# plt.plot(np.max(np.abs(y), axis=1)[:10000])
# plt.plot(y[:,8])
# plt.plot(np.median(np.abs(y), axis=1)[:])
plt.plot(np.median(np.abs(y), axis=1)[:]+1e-17)
plt.yscale('log')
# plt.ylim(1e-16, 1e-13)

In [ ]:
plt.plot(new_data['Cce']['CceTimeStep.dat'][:,1:], label='CceTimeStep')

In [ ]:
with h5py.File(WT_data, "r") as outfile:
    for key1, item1 in outfile.items():
        print(key1, item1.shape)

In [ ]:
extraction_radii = '9999'
create_WT_data_compressed(WT_data, Path("/media/himanshu/T7_500g/CCE_data/test")/f"new_temp_R{extraction_radii}.h5", trange=np.arange(0,2500,0.1), rand_amp=None, extraction_radii = int(extraction_radii))

In [ ]:
from pathlib import Path

In [ ]:
folder_test1 = Path("/groups/sxs/hchaudha/spec_runs/single_bh_CCE/runs/test1")
for i in folder_test1.glob("*"):
    print(i.stem)

In [ ]:
folder_rad_dep = Path("/groups/sxs/hchaudha/spec_runs/single_bh_CCE/runs/radius_dependence")
for i in folder_rad_dep.glob("*"):
    if i.stem in ['0001','0002']:
        continue
    print(i.stem)